In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import gc
import joblib 
warnings.filterwarnings("ignore")
import joblib
import pickle
from tqdm import tqdm
import random 
import lightgbm as lgb

from xgboost import XGBRegressor

In [2]:
sales = pd.read_csv('sales_train_evaluation.csv')
sales.name = 'sales'
calendar = pd.read_csv('calendar.csv')
calendar.name = 'calendar'
prices = pd.read_csv('sell_prices.csv')
prices.name = 'prices'


In [3]:
print(sales.shape)
print(calendar.shape)
print(prices.shape)

(30490, 1947)
(1969, 14)
(6841121, 4)


In [ ]:
sales.head()

In [ ]:
sales.isna().sum().sum()

In [ ]:
sales['cat_id'].unique()

In [ ]:
sales['dept_id'].unique()

In [ ]:
sales['store_id'].unique()

In [ ]:
sales['store_id'].value_counts()

In [ ]:
prices['item_id'].nunique()

In [ ]:
#Add zero sales for the remaining days 1942-1969
for d in range(1948,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)
sales.shape

In [ ]:
sales.columns

In [ ]:
calendar.head()

In [ ]:
calendar.isna().sum().sum()

In [ ]:
calendar['event_type_1'].fillna('No event1',inplace = True)

In [ ]:
calendar['event_type_2'].fillna('No event2',inplace = True)

In [ ]:
calendar.tail()

In [ ]:
prices.head(10)

In [ ]:
prices.isna().sum().sum()

In [4]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [8]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
             var_name='d', value_name='sold').dropna()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [9]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [10]:
df.shape

(59181090, 22)

In [11]:
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')

In [ ]:
df.shape

As we have seen stats based model does not perform well on our data, we will do featture engineering and apply machine learning based Algoritham 

Feature engineering based on Above EDA .

In [13]:
from sklearn.preprocessing import LabelEncoder
def label_encoding(train, feature):    
    encoder = LabelEncoder()
    train[feature] = encoder.fit_transform(df[feature])
    
    return train[feature]

In [14]:
df['id']  = label_encoding(df,"id" )
df['dept_id']  = label_encoding(df,"dept_id" )
df['cat_id'] = label_encoding(df,"cat_id" )
df['store_id']  = label_encoding(df,"store_id" )
df['state_id']  = label_encoding(df,"state_id" )
df['event_type_1']  = label_encoding(df,"event_type_1" )
df['event_type_2']  = label_encoding(df,"event_type_2" )
df['event_name_1']  = label_encoding(df,"event_type_1" )
df['event_name_2']  = label_encoding(df,"event_type_2" )
df['weekday']  = label_encoding(df,"weekday" )
df['item_id']  = label_encoding(df,"item_id" )
df['wm_yr_wk']  = label_encoding(df,"wm_yr_wk" )


In [ ]:
df.info()

In [15]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int32         
 1   item_id       int32         
 2   dept_id       int32         
 3   cat_id        int32         
 4   store_id      int32         
 5   state_id      int32         
 6   d             int16         
 7   sold          int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int64         
 10  weekday       int32         
 11  wday          int8          
 12  month         int8          
 13  year          int16         
 14  event_name_1  int64         
 15  event_type_1  int64         
 16  event_name_2  int64         
 17  event_type_2  int64         
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
 21  sell_price    float16       
dtypes: datetime64[ns](1), float16(1), int16(3), int32(7), int64(5), 

In [17]:
shifting = 28 #shift period in order to account for 28 days to forecast
df['lag_'+str(shifting)] = df.groupby('id')['sold'].shift(shifting).astype(np.float16)

In [18]:
#Introduce lags
lags = [7,14]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['lag_28'].shift(lag).astype(np.float16).fillna(0)

In [19]:

df['state_sold_avg'] = df.groupby('state_id')['lag_28'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['lag_28'].transform('mean').astype(np.float16)
df['cat_sold_avg'] = df.groupby('cat_id')['lag_28'].transform('mean').astype(np.float16)
df['cat_daily_avg'] = df.groupby(['weekday','cat_id'])['lag_28'].transform('mean').astype(np.float16)
df['cat_monthly_avg'] = df.groupby(['month','cat_id'])['lag_28'].transform('mean').astype(np.float16)
df['cat_dept_avg'] = df.groupby(['cat_id','dept_id'])['lag_28'].transform('mean').astype(np.float16)
df['cat_dept_daily_sold_avg'] = df.groupby(['weekday','dept_id','cat_id'])['lag_28'].transform('mean').astype(np.float16)
df['cat_dept_monthly_avg'] = df.groupby(['month','dept_id','cat_id'])['lag_28'].transform('mean').astype(np.float16)


In [20]:
df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['lag_28'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16).fillna(0)

In [21]:
df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['lag_28'].transform(lambda x: x.expanding(2).mean()).astype(np.float16).fillna(0)

In [23]:
df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['lag_28'].transform('mean').astype(np.float16)
df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['lag_28'].transform('mean').astype(np.float16)
df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)

In [24]:
df['sell_price'] = df['sell_price'].interpolate(method='linear', inplace=True)

#we left with 7 missing values after filling with interpolate, so fill with 0
df['sell_price'] = df['sell_price'].fillna(0) 

In [25]:
for day in range(1942,1970):
  sales['d_' + str(day)] = 0
  sales['d_' + str(day)] = sales['d_' + str(day)].astype(np.int16)

In [26]:
df.to_pickle('data.pkl')
del df
gc.collect();


In [5]:
df = pd.read_pickle('data.pkl')


In [6]:
df.drop(['date','wday'],axis=1,inplace=True)

In [7]:
X_train = df[df["d"] < 1914].drop("sold", axis=1)
X_val =  df[(df['d']>=1914) & (df['d']<=1941)].drop('sold',axis=1)
X_test = df[df["d"] >= 1942].drop("sold", axis=1)

y_train = df[df["d"] < 1914]["sold"]
y_val = df[(df['d']>=1914) & (df['d']<=1941)]["sold"]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 34 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   id                       int32  
 1   item_id                  int32  
 2   dept_id                  int32  
 3   cat_id                   int32  
 4   store_id                 int32  
 5   state_id                 int32  
 6   d                        int16  
 7   sold                     int16  
 8   wm_yr_wk                 int64  
 9   weekday                  int32  
 10  month                    int8   
 11  year                     int16  
 12  event_name_1             int64  
 13  event_type_1             int64  
 14  event_name_2             int64  
 15  event_type_2             int64  
 16  snap_CA                  int8   
 17  snap_TX                  int8   
 18  snap_WI                  int8   
 19  sell_price               int64  
 20  lag_28                   float16
 21  sold_l

In [9]:
df.shape

(59181090, 34)

In [10]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,...,dept_sold_avg,cat_sold_avg,cat_daily_avg,cat_monthly_avg,cat_dept_avg,cat_dept_daily_sold_avg,cat_dept_monthly_avg,rolling_sold_mean,expanding_sold_mean,selling_trend
0,14370,1437,3,1,0,0,1,0,0,2,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
1,14380,1438,3,1,0,0,1,0,0,2,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
2,14390,1439,3,1,0,0,1,0,0,2,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
3,14400,1440,3,1,0,0,1,0,0,2,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
4,14410,1441,3,1,0,0,1,0,0,2,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN


In [11]:
X_train.shape

(58327370, 33)

In [12]:
X_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,wm_yr_wk,weekday,month,...,dept_sold_avg,cat_sold_avg,cat_daily_avg,cat_monthly_avg,cat_dept_avg,cat_dept_daily_sold_avg,cat_dept_monthly_avg,rolling_sold_mean,expanding_sold_mean,selling_trend
0,14370,1437,3,1,0,0,1,0,2,1,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
1,14380,1438,3,1,0,0,1,0,2,1,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
2,14390,1439,3,1,0,0,1,0,2,1,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
3,14400,1440,3,1,0,0,1,0,2,1,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN
4,14410,1441,3,1,0,0,1,0,2,1,...,0.703125,0.566895,0.695801,0.555664,0.703125,0.871094,0.681152,0.0,0.0,NaN


In [13]:
prices["id"] = prices["item_id"].astype(str) + "_" + prices["store_id"].astype(str) + "_evaluation"
calendar["d"] = calendar["d"].apply(lambda a: int(a.split("_")[1]))

In [14]:
#https://www.kaggle.com/qcw171717/other-naive-forecasts-submission-score/notebook

for day in tqdm(range(1886, 1914)):  
    wk_id = list(calendar[calendar["d"]==day]["wm_yr_wk"])[0]
    wk_price = prices[prices["wm_yr_wk"]==wk_id]
    df_sales = sales.merge(wk_price[["sell_price", "id"]], on=["id"], how='inner')
    df_sales["unit_sales_" + str(day)] = df_sales["sell_price"] * df_sales["d_" + str(day)]
    df_sales.drop(columns=["sell_price"], inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:29<00:00,  1.05s/it]


In [15]:
col = [a for a in df_sales.columns if a.find("unit_sales")==0]
df_sales["sales"] = df_sales[col]
df_sales["weight"] = df_sales["sales"] / df_sales["sales"].sum()
df_sales.drop(columns=["sales", col[0]], axis=1, inplace=True)
df_sales["weight"] /= 12

In [16]:
aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [17]:
#function to calculate rmsse 

h = 28
n = 1913

def RMSSE(ground_truth, forecast, train_series):
    
    num = ((ground_truth - forecast)**2).sum(axis=1)
    den = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=1)
    rmsse = (1/h * num/den) ** 0.5

    return rmsse

In [18]:
def hyperparameter_tuning(X_train, y_train, model):

    model.fit(X_train, y_train)

    for d in range(1914, 1942):
        df_sales['F_' + str(d)] = model.predict(X_val[X_val['d']==d])
    
    data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
    data = data.sum()

    aggregated_df = pd.DataFrame(data).transpose()    
    aggregated_df["level"] = 1
    aggregated_df["weight"] = 1/12    
    columns = aggregated_df.columns  

    for lev in aggregation_level:
        df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
        df_t["level"] = lev
        aggregated_df = aggregated_df.append(df_t[columns])     

    train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
    actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
    forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]    

    ground_truth_df = np.array(df_sales[actual_value_columns])
    forecast_df = np.array(df_sales[forecast_value_columns])
    train_series_df = np.array(df_sales[train_columns])

    ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
    forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
    train_series_agg_df = np.array(aggregated_df[train_columns])

    df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
    aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

    df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
    aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]

    print(df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum())
    
    return (df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum())

In [19]:
%%time
wrmsse=[]
alpha_lst = [10,15,20]
lr_rate=.01
for alpha in tqdm(alpha_lst):


    xgb_model=XGBRegressor(n_estimators=10,learning_rate=lr_rate,n_jobs=-1)
    
    WRMSSE = hyperparameter_tuning(X_train, y_train, xgb_model)

    wrmsse.append(WRMSSE)


 33%|██████████████████████████                                                    | 1/3 [1:07:41<2:15:23, 4061.82s/it]

3.452178916309831


 67%|████████████████████████████████████████████████████                          | 2/3 [2:17:39<1:09:01, 4141.83s/it]

3.452178916309831


100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [3:30:50<00:00, 4216.91s/it]

3.452178916309831
Wall time: 3h 30min 50s


In [20]:
minpos = wrmsse.index(min(wrmsse))
alpha=alpha_lst[minpos]

In [21]:
#fit the model on best parameters

m_xgb= XGBRegressor(n_estimators=alpha,lr_rate=.01 ,n_jobs=-1)

m_xgb.fit(X_train, y_train)

[09:32:02] WARNING: ..\src\learner.cc:576: 
Parameters: { "lr_rate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             lr_rate=0.01, max_delta_step=0, max_depth=6, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=10, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [22]:
with open('m_xgb_1', 'wb') as file:
        pickle.dump(m_xgb, file)

In [24]:
for d in range(1914, 1942):
    df_sales['F_' + str(d)] = m_xgb.predict(X_val[X_val['d']==d])

In [25]:
def WRMSSE(df_sales):

  aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

  data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
  data = data.sum()

  aggregated_df = pd.DataFrame(data).transpose()    
  aggregated_df["level"] = 1
  aggregated_df["weight"] = 1/12    
  columns = aggregated_df.columns  

  for lev in aggregation_level:
      df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
      df_t["level"] = lev
      aggregated_df = aggregated_df.append(df_t[columns])     

  #print(df_sales.shape[0], aggregated_df.shape[0], df_sales.shape[0] + aggregated_df.shape[0])
  #print(aggregated_df["weight"].sum() + df_sales["weight"].sum())    

  train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
  actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
  forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]    


  ground_truth_df = np.array(df_sales[actual_value_columns])
  forecast_df = np.array(df_sales[forecast_value_columns])
  train_series_df = np.array(df_sales[train_columns])

  ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
  forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
  train_series_agg_df = np.array(aggregated_df[train_columns])

  df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
  aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

  df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
  aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]

  print("df", df_sales["wrmsse"].sum())
  print("agg_df",aggregated_df["wrmsse"].sum())

  WRMSSE = df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum()
  #print(WRMSSE)

  return WRMSSE 

In [26]:
WRMSSE(df_sales)

df 0.08944588827619057
agg_df 0.8243029284315251


0.9137488167077157

In [ ]:
import xgboost as xgb
xgb_model=xgb.XGBRegressor(n_estimators=10,learning_rate=.01,)
xgb_model.fit(X_train, y_train)

In [ ]:
model = RandomForestRegressor(n_estimators=10, max_depth=15,n_jobs=-1)
model.fit(X_train, y_train)